#Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Teste com apenas Shooting em um ano

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
data = requests.get(standings_url, headers=headers)

In [ ]:
soup = BeautifulSoup(data.text)
print(soup)
standings_table = soup.select("table.stats_table")[0]
print(standings_table)
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
team_url = [f"https://fbref.com{l}" for l in links]
team_url

In [ ]:
team_url = team_url[0]
data = requests.get(team_url)
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
matches

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and "all_comps/shooting/" in l]
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on= "Date")

#Iterando em anos

In [ ]:
years = list(range(2022,2020,-1))
years

In [ ]:
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if "/squads/" in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        data = requests.get(team_url)
        print(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        #print(shooting)
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on= "Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")

In [8]:
#Code using functions, iterating from 2023 to 2018 and for all teams

In [3]:
def get_team_urls(standings_url):
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if "/squads/" in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    return team_urls


In [4]:
def get_previous_season_url(current_season_url):
    data = requests.get(current_season_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    previous_season = soup.select("a.prev")[0].get("href")
    return f"https://fbref.com{previous_season}"

In [5]:
def fetch_and_process_team_data(team_url, year, end_link):
    try:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        #print(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and end_link in l]

        team_data = pd.DataFrame()

        if links:
            data = requests.get(f"https://fbref.com{links[0]}")
            
            if end_link == "all_comps/shooting/":
                shooting = pd.read_html(data.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                print("shooting")
                try:
                    team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")            
                except ValueError:
                    pass
            if end_link == "all_comps/keeper/":
                goalkeeper = pd.read_html(data.text, match="Goalkeeping")[0]
                goalkeeper.columns = goalkeeper.columns.droplevel()
                print("keeper")
                try:
                    team_data = matches.merge(goalkeeper[["Date", "Saves", "Save%", "CS", "PKatt", "PKA", "PKsv", "PKm"]], on="Date")
                except ValueError:
                    pass

            if end_link == "all_comps/passing/":
                passing = pd.read_html(data.text, match="Passing")[0]
                passing.columns = passing.columns.droplevel()
                print("passing")
                try:
                    team_data = matches.merge(passing[["Date", "Cmp", "Att", "Cmp%", "TotDist", "PrgDist"]], on="Date")
                except ValueError:
                    pass

            if end_link == "all_comps/possession/":
                possession = pd.read_html(data.text, match="Possession")[0]
                possession.columns = possession.columns.droplevel()
                print("possession")
                try:
                    team_data = matches.merge(possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Att Pen", "Live"]], on="Date")
                except ValueError:
                    pass

        else:
            print("No matching links found.")

        team_data["Season"] = year
        team_data["Team"] = team_name
        time.sleep(5)
        return team_data

    except Exception as e:
        print(f"Error fetching data for {team_url}: {e}")
        return None

In [6]:
def main(years, initial_standings_url):
    all_matches = []
    links_list = ["all_comps/shooting/", "all_comps/keeper/", "all_comps/passing/", "all_comps/possession/"]

    standings_url = initial_standings_url
    while standings_url:
        print(f"Processing standings URL: {standings_url}")
        if standings_url.find(years) != -1:
            break
        else:
            team_urls = get_team_urls(standings_url)

            if not team_urls:
                print("No team URLs found.")
                break

            for team_url in team_urls:
                team_data_list = []  # Create a list to store data for each type of statistic
                for end_link in links_list:
                    team_data = fetch_and_process_team_data(team_url, years.replace("/", ""), end_link)

                    if team_data is not None:
                        # Reset the column names for each type of statistic DataFrame
                        team_data.columns = [f"{end_link[:-1]}_{c}" if c != 'Date' else c for c in team_data.columns]
                        team_data_list.append(team_data)

                if team_data_list:
                    # Concatenate data for different types of statistics for the same team
                    team_data_combined = pd.concat(team_data_list, axis=1)
                    all_matches.append(team_data_combined)

            standings_url = get_previous_season_url(standings_url)

    return pd.concat(all_matches, ignore_index=True)

In [7]:
if __name__ == "__main__":
    years = "/2017-2018/2017-2018"
    initial_standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"  # Initial URL

    result = main(years, initial_standings_url)
    print(result)
    result.columns = [c.lower() for c in result.columns]
    result.to_csv("matches.csv")

Processing standings URL: https://fbref.com/en/comps/9/Premier-League-Stats
shooting


KeyboardInterrupt: 